# Opening Detector

General Idea: Create a tree-like structure where Nodes are openings, and Edges are moves. For readability sake I use the term `continuation` instead of `edge`.

Store the directed-graph-like structure in a pickle file.

When detecting, load the search-tree-like structure in a class and use it to navigate through the moves to find the right opening name.

Use the data here: https://github.com/lichess-org/chess-openings to create the nested-dict-like structure.

In [1]:
import pickle

In [6]:
class OpeningNode:
    def __init__(self, moveorder: list, name=None):
        self.name = name
        self.continuations = dict()
        self.moveorder = moveorder

    def has_continuation(self, move):
        return move in self.continuations.keys()
        
    def get_or_add_continuation(self, move):
        if move not in self.continuations.keys():
            self.continuations[move] = OpeningNode(self.moveorder + [move])
        return self.continuations[move]
            
        
    def __str__(self):
        string_representation = f"{self.name}\n"
        for key in self.continuations:
            string_representation = f"{string_representation}{key}:{self.continuations[key].__str__()}\n" 
        return string_representation

### Looping over files, rows, and moves

In [7]:
import pandas as pd
from os import listdir
from tqdm.notebook import tqdm

In [8]:
from src.jchess.pgn.parser import openingnotation

In [9]:
%%time
openings = 0
directory = "/home/jaco/Python/chess-openings/" # Clone of https://github.com/lichess-org/chess-openings
root = OpeningNode([], "root")
for file in tqdm([x for x in listdir(directory) if x.endswith('.tsv')]):
    df = pd.read_csv(f"{directory}{file}", sep='\t')
    for index, row in df.iterrows():
        current_node = root
        moves = openingnotation.parse(row['pgn']).or_die()
        for move in moves:
            current_node = current_node.get_or_add_continuation(move)
        current_node.name = row['name']
        openings += 1
print(f"Openings: {openings}")


  0%|          | 0/5 [00:00<?, ?it/s]

Openings: 3373
CPU times: user 793 ms, sys: 3.21 ms, total: 796 ms
Wall time: 794 ms


In [13]:
with open("data/opening_graph.pickle", 'wb+') as f:
    pickle.dump(root, f)

Let's find the opening of `1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6 5. a4 a6 6. b3 b7` in `root`

As you can see, it's the Amsterdam Attack, but with an extra couple of moves.

The idea is that we will traverse the graph-like-tree-dictionary until there's no continuation.

In [11]:
with open("data/opening_graph.pickle", 'rb') as f:
    tree_like_dictionary_graph = pickle.load(f)

In [12]:
op = "1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6 5. a4 a6 6. b3 b7"
moves_in_opening = openingnotation.parse(op).or_die()
c_node = tree_like_dictionary_graph
for move in moves_in_opening:
    if c_node.has_continuation(move):
        c_node = c_node.continuations[move]
    else:
        break
opening = c_node.name
print(opening)

Amsterdam Attack
